<a href="https://colab.research.google.com/github/AkashCreator/Pyspark/blob/main/Pyspark_Football.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [96]:
# install pyspark
!pip install pyspark

In [97]:
# read dataset using Sparksession
from pyspark.sql import SparkSession
spark=SparkSession.builder.appName("football").getOrCreate()

In [98]:
#display spark session
spark

In [99]:
#read football main dataset and show stats
full_data=spark.read.csv('./football_analysis/FullData.csv',header=True,inferSchema=True)
full_data.summary().show()

+-------+----------------+-----------+-----------------+-----------------+----------------+-------------+------------------+------------+------------------+-----------------+------+------+--------------+----------+------------------+------------------+---------------+------------------+------------------+------------------+------------------+-----------------+------------------+------------------+------------------+-----------------+------------------+-----------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+-----------------+------------------+------------------+-----------------+-----------------+-----------------+------------------+------------------+------------------+
|summary|            Name|Nationality|National_Positio

# Below are some of the observations made after running summary command

  
1.   There is one entry where we dont have data for Club_Position and some other parameters
2.   Minimum age of the player is 17(kinda expected) and max age is 47 which is bit odd
3.   Minimum weight is coming as 100 kg and Maximum weight is coming as 99 kg which is absurd(so more analysis is required here to identify rootcause)
4.   Contract expiry year is coming as Float so needs some formatting

In [100]:
#read sample data
full_data.sample(withReplacement=False,fraction=0.1).show()

+-------------------+-----------+-----------------+------------+---------------+-------------+--------+------------+---------------+------+------+------+--------------+----------+---+------------------+---------------+---------+-----------+------------+---------+-------+--------------+---------------+----------+---------+------------------+-------------+------+---------+--------+----------+---------+------------+-----+-------+--------+-------+-------+-------+-------+----------+---------+----------+-----+-----------------+---------+-------+--------------+---------+----------+-----------+-----------+
|               Name|Nationality|National_Position|National_Kit|           Club|Club_Position|Club_Kit|Club_Joining|Contract_Expiry|Rating|Height|Weight|Preffered_Foot|Birth_Date|Age|Preffered_Position|      Work_Rate|Weak_foot|Skill_Moves|Ball_Control|Dribbling|Marking|Sliding_Tackle|Standing_Tackle|Aggression|Reactions|Attacking_Position|Interceptions|Vision|Composure|Crossing|Short_Pass|L

In [101]:
#display schema
full_data.dtypes

[('Name', 'string'),
 ('Nationality', 'string'),
 ('National_Position', 'string'),
 ('National_Kit', 'double'),
 ('Club', 'string'),
 ('Club_Position', 'string'),
 ('Club_Kit', 'double'),
 ('Club_Joining', 'string'),
 ('Contract_Expiry', 'double'),
 ('Rating', 'int'),
 ('Height', 'string'),
 ('Weight', 'string'),
 ('Preffered_Foot', 'string'),
 ('Birth_Date', 'string'),
 ('Age', 'int'),
 ('Preffered_Position', 'string'),
 ('Work_Rate', 'string'),
 ('Weak_foot', 'int'),
 ('Skill_Moves', 'int'),
 ('Ball_Control', 'int'),
 ('Dribbling', 'int'),
 ('Marking', 'int'),
 ('Sliding_Tackle', 'int'),
 ('Standing_Tackle', 'int'),
 ('Aggression', 'int'),
 ('Reactions', 'int'),
 ('Attacking_Position', 'int'),
 ('Interceptions', 'int'),
 ('Vision', 'int'),
 ('Composure', 'int'),
 ('Crossing', 'int'),
 ('Short_Pass', 'int'),
 ('Long_Pass', 'int'),
 ('Acceleration', 'int'),
 ('Speed', 'int'),
 ('Stamina', 'int'),
 ('Strength', 'int'),
 ('Balance', 'int'),
 ('Agility', 'int'),
 ('Jumping', 'int'),
 ('He

First we will analyse the weight data

In [102]:
#As expected we are getting min as 100 because of kg in the column data so we will remove it to allow better analysis
full_data.select('Weight').distinct().orderBy('Weight',ascending=False).show()

+------+
|Weight|
+------+
| 99 kg|
| 98 kg|
| 97 kg|
| 96 kg|
| 95 kg|
| 94 kg|
| 93 kg|
| 92 kg|
| 91 kg|
| 90 kg|
| 89 kg|
| 88 kg|
| 87 kg|
| 86 kg|
| 85 kg|
| 84 kg|
| 83 kg|
| 82 kg|
| 81 kg|
| 80 kg|
+------+
only showing top 20 rows



In [103]:
#import sql functions from pyspark library
#replace ' kg' string to empty
#type string data to Int and then display the sorted data to ensure its formatted correctly
from pyspark.sql.functions import *
full_data=full_data.withColumn('Weight',regexp_replace('Weight',' kg',''))
full_data.withColumn('Weight',full_data.Weight.cast('Integer'))\
.select('Weight').orderBy('Weight',ascending=False)\
.head(5)

[Row(Weight=110),
 Row(Weight=110),
 Row(Weight=107),
 Row(Weight=102),
 Row(Weight=102)]

# "Above weight data confirms that formatting has been done correctly and now can be used for any analysis/aggregates"